# Домашняя работа N6. 

## Лазарев Эдуард N33471 

In [1]:
import numpy as np
import random

In [2]:
# Лазарев
text = '01100100011101010100'
text

'01100100011101010100'

Посторим матрицу адамара H_8.

In [3]:
H = np.ones((1, 1), dtype=np.int8)
while len(H) < 8:
    tmp_H = H
    H = np.zeros([len(H) * 2] * 2, dtype=np.int8)
    H[:len(tmp_H), :len(tmp_H)] = H[len(tmp_H):, :len(tmp_H)] = H[:len(tmp_H), len(tmp_H):] = tmp_H
    H[len(tmp_H):, len(tmp_H):] = tmp_H * -1

print(H)

[[ 1  1  1  1  1  1  1  1]
 [ 1 -1  1 -1  1 -1  1 -1]
 [ 1  1 -1 -1  1  1 -1 -1]
 [ 1 -1 -1  1  1 -1 -1  1]
 [ 1  1  1  1 -1 -1 -1 -1]
 [ 1 -1  1 -1 -1  1 -1  1]
 [ 1  1 -1 -1 -1 -1  1  1]
 [ 1 -1 -1  1 -1  1  1 -1]]


Напишем две функции:

* encode - Преобразует исходное сообщение в закодированное, предварительно деля его на блоки размером по четре символа.
* decode - Востанавливает сообщение из закодированного, разбивая полученное сообщения на блоки размером по восемь 
символов и умножая их на матрицу Адамара, устраняя ошибку если таковая была.

In [4]:
H = np.array([[ 1,  1,  1,  1,  1,  1,  1,  1],
                [ 1, -1,  1, -1,  1, -1,  1, -1],
                [ 1,  1, -1, -1,  1,  1, -1, -1],
                [ 1, -1, -1,  1,  1, -1, -1,  1],
                [ 1,  1,  1,  1, -1, -1, -1, -1],
                [ 1, -1,  1, -1, -1,  1, -1,  1],
                [ 1,  1, -1, -1, -1, -1,  1,  1],
                [ 1, -1, -1,  1, -1,  1,  1, -1]], dtype=np.int8)

def encode(text):
    block_size = len(H) // 2
    result = ""
    if len(text) % block_size:
        text = "0" * (block_size - len(text) % block_size) + text

    for idx in range(len(text) // block_size):
        block = text[block_size * idx: block_size * (idx + 1)]

        row = H[int(block[1:], 2)]
        if int(block[0]):
            row = -row
        
        result += "".join(map(str, (row + 1) // 2))
    return result

def decode(text):
    block_size = len(H)
    result = ""
    if len(text) % block_size:
        text = "0" * (block_size - len(text) % block_size) + text
    

    for idx in range(len(text) // block_size):
        block = text[block_size * idx: block_size * (idx + 1)]
        v = np.array(list(map(lambda x: int(x) * 2 - 1, block)))
        F = np.dot(v, H)
        max_idx = np.argmax(np.abs(F))

        result += "1" if F[max_idx] < 0 else "0"

        result += "0" * (len(bin(block_size)) - len(bin(max_idx)) - 1) + bin(max_idx)[2:]

    return result

### Проверка работы

Вызовем функцию encode для исходного текста и попытаемся востановить исходное сообщение после кодирования.
Видим что результат разкодирования совпадает с исходным текстом. А значит функция работает верно.

In [5]:
text, encode(text), decode(encode(text))

('01100100011101010100',
 '1100001111110000100101101010010111110000',
 '01100100011101010100')

Проверим возможность нахождения и исправления ошибок

Проведём десять тестов в которых будет допущенно от одной до трёх ошибок. После мы будем пытаться восстановить исходное сообщение.

In [6]:
encoded = list(encode(text))
changer = {"0": "1", "1": "0"}

for _ in range(10):
    error_positions = [random.randint(0, len(encoded) - 1) for _ in range(random.randint(1, 3))]

    print(f"Errors in positions: {error_positions}")

    error_text = encoded.copy()
    for error_pos in error_positions:
        error_text[error_pos] = changer[error_text[error_pos]]
    
    decoded = decode("".join(error_text))
    print(f"\tDecoded: {decoded}\n"
          f"\tIs right: {decoded == text}")

Errors in positions: [15, 24]
	Decoded: 01100100011101010100
	Is right: True
Errors in positions: [35]
	Decoded: 01100100011101010100
	Is right: True
Errors in positions: [0, 22, 12]
	Decoded: 01100100011101010100
	Is right: True
Errors in positions: [14, 13]
	Decoded: 01100000011101010100
	Is right: False
Errors in positions: [26, 29]
	Decoded: 01100100011110000100
	Is right: False
Errors in positions: [0]
	Decoded: 01100100011101010100
	Is right: True
Errors in positions: [20, 20, 1]
	Decoded: 01100100011101010100
	Is right: True
Errors in positions: [21, 35]
	Decoded: 01100100011101010100
	Is right: True
Errors in positions: [28, 1, 24]
	Decoded: 01100100011111000100
	Is right: False
Errors in positions: [1, 21]
	Decoded: 01100100011101010100
	Is right: True


Видим что данный метод неплохо справляется с корекцией ошибок. Но если у нас будет две или больше ошибок в одном блоке закодированного сообщения, то данный блок точно восстановить не получиться.